# Importing chartevents into Obs table

In [ ]:
import pandas as pd
import numpy as np

import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
df_chartitems = pd.read_sql("SELECT itemid, label FROM mimic4.d_items where linksto = 'chartevents' and param_type!='Checkbox' and (category='Routine Vital Signs' or category='General') and itemid IS NOT NULL and label IS NOT NULL", con = mimicconn)
df_chartitems

In [ ]:
df_chartitems['label'] = df_chartitems['label'].str.lower()
df_chartitems

In [ ]:
df_chartitems = df_chartitems.drop_duplicates(subset = 'label')
df_chartitems

In [ ]:
chartietms_dict = dict(zip(df_chartitems.itemid, df_chartitems.label))
len(chartietms_dict)

# Dealing with chartevents with numeric value

In [ ]:
df_chartevents_mimic = pd.read_sql("SELECT subject_id, hadm_id, itemid, storetime, value from mimic4.chartevents where itemid IN (SELECT distinct itemid FROM mimic4.d_items where linksto = 'chartevents' and param_type!='Checkbox' and (category='Routine Vital Signs' or category='General') and hadm_id IS NOT NULL AND value IS NOT NULL AND value NOT regexp '[[:alpha:]]')", con = mimicconn)
df_chartevents_mimic

In [ ]:
df_chartevents_mimic['label'] = df_chartevents_mimic['itemid'].map(chartietms_dict)
df_chartevents_mimic

In [ ]:
df_chartevents_mimic.isnull().sum()

In [ ]:
df_concept = pd.read_sql('select  * from `concept`', con = conn)
df_concept

In [ ]:
chartevents_concept = df_concept[107149 : 107213]
chartevents_concept

In [ ]:
chartevents_concept_dict = dict(zip(chartevents_concept.description, chartevents_concept.concept_id))
len(chartevents_concept_dict)

In [ ]:
df_chartevents_mimic['concept_id'] = df_chartevents_mimic['label'].map(chartevents_concept_dict)
df_chartevents_mimic

In [ ]:
df_chartevents_mimic['date_created']= '2022-01-01'
df_chartevents_mimic['creator'] = 1

import uuid
df_chartevents_mimic['uuid'] = [uuid.uuid4() for x in range(len(df_chartevents_mimic))]

In [ ]:
# df_chartevents_mimic

In [ ]:
df_chartevents_mimic = df_chartevents_mimic.rename({"subject_id": "person_id", "hadm_id": "encounter_id", "storetime": "obs_datetime", "value":"value_numeric" }, axis='columns')

df_chartevents_mimic = df_chartevents_mimic.drop(columns=['itemid', 'label'])

In [ ]:
# df_chartevents_mimic

In [ ]:
column_names = ['person_id', 'concept_id', 'encounter_id', 'obs_datetime', 'value_numeric', 'creator', 'date_created', 'uuid']
df_chartevents_mimic = df_chartevents_mimic.reindex(columns=column_names)


In [ ]:
df_chartevents_mimic['value_numeric']=pd.to_numeric(df_chartevents_mimic.value_numeric, errors='coerce')

In [ ]:
df_chartevents_mimic.isnull().sum()

# Importing chartevents with numeric value to obs

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_chartevents_mimic.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_chartevents_mimic.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

# Dealing with chartevents with value Text

In [ ]:
df_chartevents_mimic_text = pd.read_sql("SELECT subject_id, hadm_id, itemid, storetime, value from mimic4.chartevents where itemid IN (SELECT distinct itemid FROM mimic4.d_items where linksto = 'chartevents' and param_type!='Checkbox' and (category='Routine Vital Signs' or category='General') and hadm_id IS NOT NULL AND value IS NOT NULL AND value regexp '[[:alpha:]]')", con = mimicconn)
df_chartevents_mimic_text

In [ ]:
df_chartevents_mimic_text['label'] = df_chartevents_mimic_text['itemid'].map(chartietms_dict)
df_chartevents_mimic_text

In [ ]:
df_chartevents_mimic_text['concept_id'] = df_chartevents_mimic_text['label'].map(chartevents_concept_dict)
df_chartevents_mimic_text

In [ ]:
df_chartevents_mimic_text['date_created']= '2022-01-01'
df_chartevents_mimic_text['creator'] = 1

import uuid
df_chartevents_mimic_text['uuid'] = [uuid.uuid4() for x in range(len(df_chartevents_mimic_text))]

In [ ]:
df_chartevents_mimic_text = df_chartevents_mimic_text.rename({"subject_id": "person_id", "hadm_id": "encounter_id", "storetime": "obs_datetime", "value":"value_text" }, axis='columns')
df_chartevents_mimic_text = df_chartevents_mimic_text.drop(columns=['itemid', 'label'])

In [ ]:
df_chartevents_mimic_text

In [ ]:
column_names = ['person_id', 'concept_id', 'encounter_id', 'obs_datetime', 'value_text', 'creator', 'date_created', 'uuid']
df_chartevents_mimic_text = df_chartevents_mimic_text.reindex(columns=column_names)

In [ ]:
df_chartevents_mimic_text

# Importing chartevents with value text to obs table

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_chartevents_mimic_text.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_chartevents_mimic_text.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 